# A GIS-based framework that identifies priority areas for stormwater management near MS4 sewer sites in Washington, DC.
By Jonathan Kvilhaug and Lucy Meyer

Version 1.1 (December 2019)

---------------------------------------------------------------------------------------------------------

## Objective: 

The pollution of urban rivers in Washington DC is classifiable by two distinct sources and their corresponding infrastructure. DC, like many urban areas, has a large share of impervious surfaces which prevent the infiltration of rainwater during storms. As a consequence, large volumes of rainwater have the ability to accumulate during and after storms which overwhelms existing infrastructure and results in the release of mixed excess stormwater and raw sewage to the Potomac River and its tributaries. The objective of this study is to develop a simplified tool that identifies water pollution severity at sewer outflow sites. The model will assess sewer outflow points across DC and provide a weighted multi-criteria analysis of at-risk areas based on watershed, sewershed, and vulnerable communities. Moreover, this analysis is aggragated at the Census Tract scale.

---------------------------------------------------------------------------------------------------------


## A. Data Collection

### 1. Analysis Data for Multi-Criteria Evaluation (MCE)
- The *Washington DC 2010 Census Tract* shapefile [^1] will be used as the primary geography for this study. Utilization of median income by census tract attribute in a MCE and in classifying symbology for display. Additional demographic data is included.

- The *Wards of Washington DC* [^2] will be utilized to classify census tracts by their larger municipal groupings, an effective approximate for socio-economic needs. 

- The *Watersheds of Washington DC* are an important evaluative criteria to prioritize intervention in Census Tracts of the MS4. [^3]

- The *Best Management Practices* indicate green infrastructure projects that work to reduce the volume of stormwater runoff that accumulate in the urban landscape. [^4]

### 2. Supplementary Data for Cartographical Analysis

- Geographic data on the extent of *Municipal Separate Storm Sewer Systems* (MS4) [^5] and *Combined Sewer Systems* (CSS) [^6] show the existing stormwater infrastructure and where green infrastructure should be utilized. The CSS shapefile will be used to produce final maps, thus, is for display purposes only.

- The following shapefiles are incorporated for display purposes only to better contextualize later visualizations. The *Municipal boundary of Washington DC* [^7]; the *Waterbodies of the District of Columbia* [^8]; and the *National Parks of the District of Columbia* [^9]. 

***
[^1]: (Source: 2010 Census Tract, US Census Bureau, Open Data DC)

[^2]: (Source: Wards from 2012, DC Office of Planning, Open Data DC)

[^3]: (Source: Watersheds clipped to DC, DC DOEE, Open Data DC)

[^4]: (Source: Best Management Practices, DC DOEE, Open Data DC)

[^5]: (Source: MS4 Sewer Shed, DC DOEE, Open Data DC)

[^6]: (Source: CSS Sewer Boundary, DC DOEE, Open Data DC)

[^7]: (Source: Washington DC Boundary, DC Office of Planning, Open Data DC)

[^8]: (Source: Waterbodies, DC DOEE, Open Data DC

[^9]: (Source: National Parks, National Parks Service, Open Data DC)

***

In [0]:
# Import modules
import geopandas as gpd
import pandas as pd
%matplotlib inline

In [0]:
# Set path
# Path changes - make sure you have a file folder with all the data or import will not work
path = '/Users/LucyMeyer/Desktop/Programming/Proj_data/' # Set path to your own folder

In [0]:
# Import raw analysis data
BMP = path +'BMP/' # Point
Ward = path + 'Ward_from_2012/' # Polgyon
Census = path + 'Census_Tracts_by_Median_Income__2000/' # Polygon
Watershed = path + 'Watersheds_Clipped_to_DC/' # Polygon

## B. Data Cleansing
This section organizes the raw data by removing columns with redudant information.

### 1. Ward

In [0]:
# Read data
Ward = gpd.read_file(Ward)
# Ward.head()

In [0]:
# Create new variable only including sorted columns for Ward
Ward1 = Ward.iloc[:,[1,14,48,49,70,82]] #iloc is columns you want to keep
# Ward1.head() # Confirm that data is correct 
# Ward1.info() 

### 2. BMP


In [0]:
# Read data
BMP1 = gpd.read_file(BMP)
BMP1.head()
# BMP1.info()

In [0]:
# Create new variable only including sorted columns for BMP
BMP2 = BMP1.iloc[:,[4,7,18,19,23,43,46,49]] 
# BMP2.head() # Confirm that data is correct
# BMP1.info()

### 3. Watershed

In [0]:
# Read data
Watershed = gpd.read_file(Watershed)
# Watershed.head()

In [0]:
# Create new variable only including sorted columns for watershed
Watershed1 = Watershed.iloc[:,[1,4]]
# Watershed1.head() # Confirm that data is correct 
# Watershed.info()

### 4. Census

In [0]:
# Clean Census data 
Census = gpd.read_file(Census)
# Census.head()

In [0]:
# Create new variable only including sorted columns for BMP
Census1 = Census.iloc[:,[6,7,8,25,26,27,28,29,30,31,53,59,60]]
# Census1.head() # Confirm that data is correct
# Census1.info()

## C. Spatial Joins
Here, spatial joins are performed between datafiles. More data cleansing occurs to remove and rename columns.

### 1. Join watershed to census
Join based on geometry (spatial location).

In [0]:
# Only retain geometry of right index aka census
Census_join = gpd.sjoin(Watershed1, Census1, how = "right", op = "intersects")
Census_join.head() # Make sure join worked correctly -- remove index_left

In [0]:
# Drop redundant columns
Census_join = Census_join.drop(['index_left'], axis = 1)

In [0]:
# rename joined columns
Census_join.rename(columns = {'NAME_x':'Watershed', 'NAME_y':'Tract Name'}, inplace = True) 
Census_join.head()
# Census_join.to_file("Census_join.shp") # Export to make sure attribute data is correct

### 2. Join ward to joined watershed-census data
Join based on geometry (spatial location).

In [0]:
# Only retain geometry of right index aka census
Census_join1 = gpd.sjoin(Ward1, Census_join, how = "right", op = "intersects")
Census_join1.head() # Make sure join worked correctly

In [0]:
# drop redundant columns
Census_join2 = Census_join1.drop(['index_left'], axis=1)
Census_join2.head()
# Census_join2.to_file("Census_join2.shp") # Export to make sure attribute data is correct

### 3. Join BMP to joined ward-watershed-census data
Join based on geometry (spatial location), then performs attribute join recieve count of BMPs per Tract (for future analysis).

In [0]:
# Copy column for Tract Name because it gets deleted when join is performed
Census_join2["Tract_Name_Copy"] = Census_join2["Tract Name"]
Census_join2.head()

In [0]:
# Perform join between BMP and joined census data
Census_join3 = gpd.sjoin(BMP2, Census_join2, how = "right", op = "within")
Census_join3.head()
# Census_BMP.to_file("Census_BMP.shp") #Export shp to double check that join is correct

In [0]:
# drop redundant columns
Census_join4 = Census_join3.drop(['WARD_y', 'index_left'], axis=1)
Census_join4.head()

# rename columns
Census_join4.rename(columns = {'WARD_x':'WARD', 'Tract_Name_Copy':'TRACT_NAME'}, inplace = True) 
Census_join4.head()

In [0]:
# Generate BMP count per census tract
grouped = Census_join4.groupby("TRACT_NAME").size()
Census_join5 = grouped.to_frame().reset_index()
Census_join5.columns = ["TRACT_NAME",'BMP Count']
Census_join5.head()

In [0]:
# Perform attribute join
Census_join6 = Census_join4.merge(Census_join5, on='TRACT_NAME')
Census_join6.head()
# Census_join6.to_file('Census_join6.shp')

## D. Multi-Criteria Evaluation

The objective of this analysis is to identify the census tracts that are best suited for green infrastructure intervention based on the geographic ecological, social, and economic conditions.

1. The first significant evaluation is to determine whether a census tract has an outflow site (MS4) located within its geography, resulting in a binary score (0, 1).
2. The second evaluation is to evaluate the contributing watershed, prioritizing intervention in the historically polluted and smaller watersheds first. (Anacostia 1; Rock Creek .75; Potomac .5)
3. The third evaluation is by the political geography of Washington DC by its Municipal Wards. (Ward 8 = 1.0; Ward 7 = 1.0; Ward 5 = .8; Ward 4 = .6; Ward 3 = .4)

In [0]:
Census_tract1 = Census_join6

In [0]:
# Census_tract1.info()
# WARD is an object so must convert to float
Census_tract1['WARD'].astype(str)

In [0]:
# Function for MCE
def MCE():

    MCE = Census_tract1
    
    def BMP_filter(sewershed):
        if (sewershed["SEWERSHED"]=="MS4"):
            return 1.0
        else:
            return 0.0

    def Watershed_filter(watershed):
        if (watershed["Watershed"]=="Anacostia River"):
            return 1.0
        if (watershed["Watershed"]=="Rock Creek"):
            return .75
        if (watershed["Watershed"]=="Potomac River"):
            return .5
        else:
            return 0
    
    def Ward_filter(ward):
        if (ward["WARD"]=="7"):
            return 1.0
        if (ward["WARD"]=="8"):
            return 1.0
        if (ward["WARD"]=="5"):
            return .8
        if (ward["WARD"]=="4"):
            return .6
        if (ward["WARD"]=="3"):
            return .4
        else: 
            return 0.0
        
    # Create new column applying filter functions
    MCE['BMP_SCORE'] = MCE.apply(BMP_filter, axis=1)
    MCE['WATERSHED_SCORE'] = MCE.apply(Watershed_filter, axis=1)
    MCE['WARD_SCORE'] = MCE.apply(Ward_filter, axis=1)
    
    def MCE_sum(row):
        return row['BMP_SCORE'] + row['WATERSHED_SCORE'] + row['WARD_SCORE']
    
    MCE['MCE_SCORE_SUM'] = MCE.apply(MCE_sum, axis=1)
    
    def MCE_score(row):
        return row['MCE_SCORE_SUM'] / 3 # 3 is the total number of scores
    
    MCE['MCE_ACTUAL_SCORE'] = MCE.apply(MCE_score, axis=1)
 
    return MCE

In [0]:
# Export MCE to file and visualize using mapping software
MCE = MCE()
MCE.to_file('MCE.shp')

##E. Data Visualization
### *1. Visualizing Existing Stormwater Infrastructure in Washington DC*
Display the Washington DC political boundary, then overlay the Combined Sewer System (CSS), followed by the Municipal Separate Storm Sewer System (MS4) overlayed above that, and finally the extent of DC Parks. The subsequent display demonstrates the relevant geography within Washington DC where Green Infrastructure projects should be implemented. Namely, within the extent of the city with MS4 infrastructure that is within the built environment rather than park space. 

![Existing Stormwater Infrastructure in Washington DC](https://drive.google.com/uc?id=1COB9vTVBWpTwL69JXMHpfzz6n9mcadMV)

### *2. Visualizing Watersheds to support targetted intervention*
Urban runoff throughout Washington DC has an outsized impact on both the Anacostia River and Rock Creek. Therefore it is critical to visualize the contributing watershed to the city's MS4 Outflow sites in order to prioritize the placement of Green Infrastructure. 

Display the Washington DC political boundary, then overlay the watersheds clipped to Washington DC, additionally display the CSS extent in order to mask the watershed where its runoff does not contribute to nutrient pollution, finally overlay the point source locations of MS4 outfalls within Washington DC. 


![Watersheds of the MS4 Sewershed](https://drive.google.com/uc?id=17CCTEtrJW8YchqNcAUUjXjwAdNyU3p8F)
---



### *3. Visualizing the political geography of suitable development*

While urban runoff pollution is sourced throughout Washington DC, it is important that impactful developments are distributed throughout the city and that the investments also reach poorer communities. 

Display the Census Tracts of the MS4 sewershed by using the Median income of each tract as the classifier of symbology. 

![Census Tracts of MS4 by Median Income](https://drive.google.com/uc?id=1qCvrgPfBIVkOpKtMlkgIXofOQKW32TOm)

###*4. Visualizing the current Green Infrastructure of Washington DC*
Throughout Washington DC there are many examples of Green Infrastructure implemented either by the city of Washington or by citizens taking advantage of incentive programs offered by the DC Department of Energy and Environment. This display demonstrates the relative distribution of Green Infrastructure. 


![Distribution of Green Infrastructure by Ward](https://drive.google.com/uc?id=1atlaF6RqDisJv_4eyfHA8J1nKGnZ0_Ye)

###*5. MCE Identifying Priority Area for Stormwater Management per Census Tract in Washington, D.C.*
Lastly, this map is the product of our analysis. It visualizes the results of the MCE score which was based on sewershed, watershed, and DC ward. MCE scores are classified into three groups. The lowest group (pale yellow), scaling between 0.17 to 0.38, represents areas within the Potomac watershed, wards 1 or 2, and may or may not fall within the MS4 sewershed. The middle ranking (orange), ranging between 0.38 and 0.72, signifies areas from a part of all watersheds, wards 3, 4, 5, and 6; and/or regions that may or may not fall within the MS4 sewershed. And finally, the highest scoring group (red) has scores between 0.72 and 1. This region highlights areas mostly within wards 7 and 8 (some from wards 4, 5, and 6); Anacostia Watershed and some from Rock Creek, and. To better interpret the impact of MCE scores on geographical analysis, it may be beneficial to classify scores into more groups. However, for the purpose of visual interpretation, it was better to delineated the scores into three groups.

![MCE of Suitable Areas for Managing Stormwater Runoff](https://drive.google.com/uc?id=1t6UZ9UlO7FKFl2SEw8rW0R6Eb507nP8x)